In [ ]:
#deal 번호를 받아와서
#http://www.tmon.co.kr/api/detail/direct/v1/detailapi/api/detail/board/review/list/4284526834?&page=100&count=100
#reviewContents가 없을 때까지

In [66]:
import urllib
from bs4 import BeautifulSoup
import requests
import json
import re

def toJson(data, file_path) :
    with open(file_path, 'w', encoding='utf-8') as file :
        json.dump(data, file)
    return 

# txt_file_path_ls = ['productUrlList_간편식_냉장_냉동.txt', 'productUrlList_과일.txt']
txt_file_path_ls = ['test.txt']

for txt_file_path in txt_file_path_ls :

    #productUrlList.txt에서 링크 리스트 불러오기
    r = open(txt_file_path, mode='rt', encoding='utf-8')
    productUrlList = r.readlines()

    review_data = []

    for prod_url in productUrlList :
        prod_idx = re.sub('http://www.tmon.co.kr/deal/','',prod_url)
        prod_idx = re.sub('\n','',prod_idx)
        
        page = 1
        while True: 
            url = "http://www.tmon.co.kr/api/detail/direct/v1/detailapi/api/detail/board/review/list/{}?&page={}&count=100".format(prod_idx, page)
            html = urllib.request.urlopen(url).read()
            soup = BeautifulSoup(html, 'html.parser')
            text = soup.get_text()
            json_data = json.loads(text)


            try :
                iter_length = len(json_data['data']['reviewContents'])

                for idx in range(iter_length) :

                    #리뷰 고유 번호
                    review_idx  = json_data['data']['reviewContents'][idx]['reviewSrl']
                    #상품 번호
                    prod_idx = deal
                    #상품 만족도
                    prod_satis = json_data['data']['reviewContents'][idx]['dealGpoint']
                    #배송 만족도
                    ship_satis = json_data['data']['reviewContents'][idx]['deliveryGpoint']
                    #구매한 옵션
                    select_options = json_data['data']['reviewContents'][idx]['buyOptionList'][0]['optionTitle']
                    #구매한 날짜
                    buy_date = json_data['data']['reviewContents'][idx]['buyOptionList'][0]['optionTitle']
                    #리뷰 작성 날짜
                    review_date = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['updateDt']
                    #사용만족도 옵션1
                    use_opt1 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['keyword']
                    use_satis1 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['selection']
                    #사용만족도 옵션2
                    use_opt2 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][1]['keyword']
                    use_satis2 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][1]['selection']
                    #리뷰 내용
                    content = json_data['data']['reviewContents'][idx]['contents']
                    #user_id
                    user_id = json_data['data']['reviewContents'][idx]['whoUpdate']
                    #memberSrl
                    memberSrl =json_data['data']['reviewContents'][idx]['memberSrl']

                    review_data.append({
                            'review_idx' : review_idx,
                            'prod_idx' : prod_idx,
                            'prod_satis' : prod_satis,
                            'ship_satis' : ship_satis,
                            'select_options' : select_options,
                            'buy_date' : buy_date,
                            'review_date' : review_date,
                            'use_opt1' : use_opt1,
                            'use_satis1' : use_satis1,
                            'use_opt2' : use_opt2,
                            'use_satis2' : use_satis2,
                            'content' : content,
                            'user_id' : user_id,
                            'memberSrl' : memberSrl
                        })
            except :
                break

            page += 1

    toJson(review_data, 'review_data_{}.json'.format(txt_file_path[15:-4]))

In [68]:
data_df = pd.read_json('review_data_.json')
data_df

,review_idx,prod_idx,prod_satis,ship_satis,select_options,buy_date,review_date,use_opt1,use_satis1,use_opt2,use_satis2,content,user_id,memberSrl
0,167871830,4284526834,5,5,04_꿀고구마 정품 대1kg (2개 주문시 5kg 발송),04_꿀고구마 정품 대1kg (2개 주문시 5kg 발송),2020-10-04 00:10:01,맛,맛있어요,신선도,신선해요,좋은품질 착한가격 감사해요,jjh*******,99016022
1,167871482,4284526834,5,5,02_★추천★꿀고구마 정품 특상1kg (2개 주문시 3kg 발송),02_★추천★꿀고구마 정품 특상1kg (2개 주문시 3kg 발송),2020-10-04 00:08:28,맛,맛있어요,신선도,신선해요,진짜 세상 쥰맛탱 개쩔탱,jjh*******,99016022
2,167867714,4284526834,1,5,05_꿀밤고구마 못난이2kg(2개 주문시 10kg 발송),05_꿀밤고구마 못난이2kg(2개 주문시 10kg 발송),2020-10-03 23:50:23,맛,아쉬워요,신선도,아쉬워요,상태가 뭐... 흠... 하나만 사보고 살걸...,red****,60097526
3,167863174,4284526834,5,5,04_꿀고구마 정품 대1kg (2개 주문시 5kg 발송),04_꿀고구마 정품 대1kg (2개 주문시 5kg 발송),2020-10-03 23:32:43,맛,보통이에요,신선도,적당해요,좋아요 만족합니당굿굿이에요,kak**********,91115010
4,167861878,4284526834,5,5,04_꿀고구마 정품 대1kg (2개 주문시 5kg 발송),04_꿀고구마 정품 대1kg (2개 주문시 5kg 발송),2020-10-03 23:27:15,맛,보통이에요,신선도,적당해요,좋아요 항상 구매하던거예요,gtc*****,42314825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6264,147033882,4284526834,3,3,a08.꿀고구마 5kg 소/한입크기,a08.꿀고구마 5kg 소/한입크기,2020-07-21 18:51:24,맛,보통이에요,신선도,적당해요,잘먹겠습니다 수고하세요,zzo***,16490873
6265,146350102,4284526834,5,5,a08.꿀고구마 5kg 소/한입크기,a08.꿀고구마 5kg 소/한입크기,2020-07-18 21:36:29,맛,보통이에요,신선도,적당해요,너무너무 좋아요 잘먹겠습니다,u03****,91143614
6266,146100750,4284526834,4,5,a01.꿀고구마 3kg 소/한입크기,a01.꿀고구마 3kg 소/한입크기,2020-07-17 20:07:44,맛,맛있어요,신선도,적당해요,상자가 적어서 3키로 아닌거 같았는데 딱 3키로네요~^^ 배송도 빠르고 맛도 좋고...,Icd*****,48886157
6267,144934546,4284526834,3,4,a08.꿀고구마 5kg 소/한입크기,a08.꿀고구마 5kg 소/한입크기,2020-07-13 17:43:12,맛,보통이에요,신선도,신선해요,맛은 맀는데 너무 가늘고 심이 많아요,lis******,84060030
